In [ ]:
%matplotlib notebook
import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display

### Hypothesis testing 

The following widgets let you select the parameters of an either one-sided or two-sided hypothesis test for the population mean. The null hypothesis is that the value of the mean is a given value. The alternate hypothesis is that the population mean is different of that value (one-sided test) or higher/lower (two-sided test).

The sampling distribution of the sample mean is close to normal, centered around the population mean, and with a standard deviation equal to the standard error of the mean. The null hypothesis is evaluated by using the Z-score of the sample mean based on that normal distribution to compute the probability of we get that sample mean by sampling from the population. This is the p-value. We reject the null hypothesis if the p-value is lower than the significance level.

In [ ]:
items_layout = widgets.Layout(flex='1 1 auto',
                      width='auto')

pm = widgets.FloatText(
        value=0,
        description='Population mean',
        layout = items_layout
    )
sm = widgets.FloatText(
        value=1,
        description='Sample mean',
        layout = items_layout
    )
ssd = widgets.FloatText(
        value=1,
        description='Sample standard deviation',
        layout = items_layout
    )
ss = widgets.FloatText(
        value=100,
        min=10,
        description='Size of the sample',
        layout = items_layout
    )

items = [
    widgets.Label('Population'),
    pm,
    widgets.Label('Sample'),
    sm,
    ssd,
    ss,
    widgets.Label('Hypothesis test'),
    
]

box_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='50%')

box = widgets.Box(children=items, layout=box_layout)

display(box)